In [1]:
import fitz  # PyMuPDF
import docx
import os
import pandas as pd

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    full_text = []

    # Extract regular paragraphs
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Extract text from tables
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                # Avoid repeating text if cells are shared
                if cell.text not in full_text:
                    full_text.append(cell.text)

    return "\n".join(full_text)



In [2]:
file_path = 'CV_Shvedyuk_Andrey_ds.pdf'  # or 'my_cv.docx'

if file_path.endswith('.pdf'):
    text = extract_text_from_pdf(file_path)
elif file_path.endswith('.docx'):
    text = extract_text_from_docx(file_path)
else:
    raise ValueError("Unsupported file type")

print(text[:1000])  # Preview first 1000 characters


Шведюк Андрей
Data Scientist
 
Москва, м. Речной вокзал

Возраст: 42 года

Цель  - получить работу Стажера или Junior Data Scientist в IT-
компании
Data Scientist (учебный и практический опыт):
• Владею программированием на Python на базовом уровне: проводил
преобразования и очистку данных, исследование зависимостей с
применением инструментов Data Science: Numpy, Pandas, Matplotlib,
Seaborn , Plotly
• Владею SQL на базовом уровне: решал задачи по загрузке и анализу
данных с применением фильтрации, агрегатных функций,
объединений таблиц
• Изучил алгоритмы и методы EDA. Проводил разведывательный
анализ, проектирование и отбор признаков, статистические тесты,
А/В тестирование
• Владею основными методами машинного обучения ML. Создавал
модели ML: регрессия, классификация, кластеризация
• Проводил валидацию данных, оценку эффективности и оптимизацию
гиперпараметров ML моделей
• Изучал линейную алгебру в контексте задач ML, математический
анализ, теория вероятностей в контексте наивного Ба

In [3]:
df = pd.DataFrame({'Filename': [os.path.basename(file_path)], 'CV_Text': [text]})
df.to_excel('extracted_cv_text.xlsx', index=False)
print("Saved to extracted_cv_text.xlsx")


Saved to extracted_cv_text.xlsx


In [2]:
import requests
import json
import pandas as pd
import os

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = "1FyhNdyIvR4hR29waKXtRf4NaGAwl5Ru"
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по разметке резюме (CV). Извлеки структурированную информацию из текста резюме.

Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.

Если какое-либо поле отсутствует в тексте — верни его как null. Если поле предполагает список (например, skills или languages), но ничего не указано — верни пустой список.

Извлеки следующие поля:
- full_name
- position
- salary_from
- salary_to
- currency
- city
- country
- email
- phone
- age
- gender
- citizenship
- relocation
- travel_ready
- work_schedule
- employment
- education_level
- education: [{institution, faculty, degree, graduation_year}]
- experience_years
- experience: [{company, position, start_date, end_date, description}]
- languages: [{language, level}]
- skills
- certifications
- achievements
- summary
- portfolio_links
- linkedin
- github
- other_links
"""

def prepare_prompt(cv_text: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст резюме:\n{cv_text}"}
    ]

def query_mistral(prompt: list):
    response = requests.post(
        API_URL,
        headers=HEADERS,
        json={
            "model": MODEL_NAME,
            "messages": prompt,
            "temperature": 0.2,
            "max_tokens": 2500
        },
        timeout=120
    )
    response.raise_for_status()
    content = response.json()["choices"][0]["message"]["content"]
    content = content.replace("```json", "").replace("```", "").strip()

    if content.lower() == "null":
        return None

    return json.loads(content)

def process_single_resume(input_path, output_path, text_column="CV_Text"):
    # Определим формат
    ext = os.path.splitext(input_path)[-1].lower()
    if ext == ".csv":
        df = pd.read_csv(input_path)
    elif ext in [".xls", ".xlsx"]:
        df = pd.read_excel(input_path)
    else:
        raise ValueError("Поддерживаются только CSV и Excel файлы")

    if text_column not in df.columns:
        raise ValueError(f"В файле нет колонки '{text_column}'")

    cv_text = df[text_column].iloc[0]  # берём первую строку
    prompt = prepare_prompt(cv_text)
    result = query_mistral(prompt)

    if result is None:
        print("Модель вернула null")
        result = {}

    # Сохраняем результат
    result_df = pd.DataFrame([result])
    
    if output_path.endswith(".xlsx"):
        result_df.to_excel(output_path, index=False)
    elif output_path.endswith(".json"):
        result_df.to_json(output_path, orient="records", force_ascii=False)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
    elif output_path.endswith(".csv"):
        result_df.to_csv(output_path, index=False)
    else:
        raise ValueError(f"Неподдерживаемый формат файла: {output_path}")

    print(f"✅ Разметка завершена. Результат сохранён в: {output_path}")


In [3]:
process_single_resume(
    input_path="extracted_cv_text.xlsx",   # файл с одним резюме
    output_path="resume_structured.json"   # 
)

✅ Разметка завершена. Результат сохранён в: resume_structured.json


In [ ]:
# это для потока резюме

import requests
import json
import pandas as pd
import time
import os

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = "YOUR_API_KEY"
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по разметке резюме (CV). Извлеки структурированную информацию из текста резюме.

Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.

Если какое-либо поле отсутствует в тексте — верни его как null. Если поле предполагает список (например, skills или languages), но ничего не указано — верни пустой список.

Извлеки следующие поля:
- full_name
- position
- salary_from
- salary_to
- currency
- city
- country
- email
- phone
- age
- gender
- citizenship
- relocation
- travel_ready
- work_schedule
- employment
- education_level
- education: [{institution, faculty, degree, graduation_year}]
- experience_years
- experience: [{company, position, start_date, end_date, description}]
- languages: [{language, level}]
- skills
- certifications
- achievements
- summary
- portfolio_links
- linkedin
- github
- other_links
"""

def prepare_prompt(cv_text: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст резюме:\n{cv_text}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.2,
                    "max_tokens": 2500
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"]
            content = content.replace("```json", "").replace("```", "").strip()

            if content.lower() == "null":
                return None

            return json.loads(content)

        except requests.exceptions.RequestException as e:
            print(f"[Ошибка запроса] Попытка {attempt + 1}: {e}")
            time.sleep(2)
        except json.JSONDecodeError as e:
            print("[Ошибка JSON]:", e)
            print("Ответ LLM:\n", content)
            return None

    return None

def process_resumes(input_file, output_file, partial_file='output_partial_cv.csv', id_column='ID'):
    df = pd.read_csv(input_file)

    if os.path.exists(partial_file):
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial[id_column])
        print(f"Найден частичный файл: {len(processed_ids)} уже обработано.")
    else:
        df_partial = pd.DataFrame()
        processed_ids = set()

    for i, row in df.iterrows():
        resume_id = row[id_column]
        if resume_id in processed_ids:
            continue

        print(f"[{i+1}/{len(df)}] Обработка резюме ID: {resume_id}")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        # Заполнить результат (пустыми или LLM-данными)
        row_result = {id_column: resume_id}
        fields = [
            "full_name", "position", "salary_from", "salary_to", "currency",
            "city", "country", "email", "phone", "age", "gender",
            "citizenship", "relocation", "travel_ready", "work_schedule",
            "employment", "education_level", "education", "experience_years",
            "experience", "languages", "skills", "certifications", "achievements",
            "summary", "portfolio_links", "linkedin", "github", "other_links"
        ]
        for field in fields:
            row_result[field] = result.get(field, None) if result else None

        df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2.5)

    df_merged = pd.merge(df, df_partial, on=id_column, how="left")
    df_merged.to_csv(output_file, index=False)
    print(f"✅ Обработка завершена. Результат сохранён в: {output_file}")
